In [36]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2 as cv 
from deepface import DeepFace
import threading
from PIL import Image
import datetime
import csv

In [37]:
def take_photo():
    cap = cv.VideoCapture(0,cv.CAP_DSHOW)
    cap.set(cv.CAP_PROP_FRAME_HEIGHT, 480)
    cap.set(cv.CAP_PROP_FRAME_WIDTH, 640)
    while True:
        try:
            _, frame = cap.read()
            cv.imshow("video", frame)
        except:
            cap.release()
            print("Something went terribly wrong!")
            break
        key = cv.waitKey(1)
        if key == ord('c'):
            cap.release()
            cv.destroyAllWindows()
            return frame
            break
        if key != -1:
            break
    cap.release()
    cv.destroyAllWindows()
    


In [38]:
# def save_img(name, img, root_folder = r'C:\\jupyter_practice\\Computer_Vision_practice\\root_folder'):
#     os.chdir(r'C:\\jupyter_practice\\Computer_Vision_practice')
#     # reference = 'nizz.png'
#     # name = input("Enter your name: ")
#     # img = cv.imread(reference)
#     os.chdir(root_folder)
#     if name not in os.listdir(root_folder):
#         os.mkdir(name)
#         to_be_appended = root_folder + r'\\' + name
#         os.chdir(to_be_appended)
#         # cv.imwrite(f'{name}{len(os.listdir('root_folder'))}', )
#         cv.imwrite(f'{name}.jpg', img)
#     elif name in os.listdir(root_folder):
#         os.chdir(root_folder + r"\\" + name)
#         cv.imwrite(name + str(len(os.listdir(root_folder + r"\\" + name))) + '.jpg', img)



def save_img(stdID,img, curr_folder = os.getcwd(), root_folder = os.path.join(os.getcwd(), 'root_folder')):
    try:
        img_folder = os.path.join(root_folder, str(stdID))
#             cv.imwrite(f'{stdID}_{len(os.listdir(os.path.join(root_folder,str(stdID))))}')
        cv.imwrite(os.path.join(img_folder, f"{stdID}_{len(os.listdir(img_folder))}.jpg"), img)

    except FileNotFoundError:
        return "can't save"

def init_folder(stdID):
    os.mkdir(os.path.join(os.getcwd(),'root_folder',str(stdID)))
                

    


In [118]:

# ref = cv.imread('sazz.jpg')
# # os.chdir(os.path.join(os.getcwd(),'root_folder','nizz'))
# cv.imwrite(os.path.join(os.getcwd(), 'root_folder', 'nizz', 'sazz_test.jpg'), ref)

In [119]:
# os.path.join(os.getcwd(),'root_folder', 'nizz', 'akdjf.jpg')

In [39]:
# def checkFace(img, root_folder = r'C:\\jupyter_practice\\Computer_Vision_practice\\root_folder'):
    
#     for folders in os.listdir(root_folder):
#         if folders.endswith('.ipynb_checkpoints'):
#             continue
#         else:
#             for images in os.listdir(root_folder + r'\\' + folders):
#                 if images.endswith('.jpg'):
#                     verification_result = DeepFace.verify(cv.imread(images), img)
#                     if verification_result[0]:
#                         return True
#                 else:
#                     continue
#     return False


# checkFace(img = 'sazz.jpg')


def checkFace(img, root_folder = r'C:\\jupyter_practice\\Computer_Vision_practice\\root_folder'):
    for folders in os.listdir(root_folder):
        if folders.endswith('.ipynb_checkpoints'):
            continue
        else:
            for images in os.listdir(root_folder + r"\\" + folders):
                if images.endswith('.jpg'):
                    try:
                        verification_result = DeepFace.verify(img, cv.imread(root_folder + r"\\" + folders + r"\\" + images))
                        if verification_result['verified']:
                            return True, folders
                    except ValueError:
                        continue
                        
    return False, None


In [57]:
class FilePointerError(Exception):
    def __init__(self, msg):
        self.msg = msg
        super().__init__(self.msg)


        
class manageEvent:
    def __init__(self, root_path, csv_name):
        self.root_path = root_path
        self.csv_name = csv_name
    
    def setHeader(self, field_names: list):
        with open(self.root_path + r'\\' +self.csv_name, mode = 'a') as fp:
            writer = csv.DictWriter(fp, fieldnames = field_names)
            if fp.tell() == 0:
                writer.writeheader()
                return field_names
            else:
                raise FilePointerError("File must be empty or the file pointer should be at start to set Header.")
    
        
    def append(self, data: dict, field_names: list, newline = ''):
        with open(self.root_path + r"\\" + self.csv_name, mode = 'a', newline = '') as fp:
            self.field_names = field_names
            writer = csv.DictWriter(fp, fieldnames = self.field_names)
            writer.writerow(data)
            
    def fetchFrom(self, csv_name, what_to_fetch: list):
        with open(self.root_path + r'\\' + csv_name) as csv_file:
            dictionary = csv.DictReader(csv_file)
            fetched_data = list()
            for rows in dictionary:
                data = dict()
                for field in what_to_fetch:
                    data[field] = rows[field]
                fetched_data.append(data)
        return fetched_data
    
    def fetchRow(self,fromCSV,header, condition):
        with open(self.root_path + r'\\' + fromCSV) as csv_file:
            reader = csv.DictReader(csv_file)
            for rows in reader:
                if rows[header] == condition:
                    return rows
                
    def fetchRowWithField(self, csv_name,fields: list, header, condition):
        dictionary_list = self.fetchFrom(csv_name, fields)
        for dictionary in dictionary_list:
            if dictionary[header] == condition:
                return dictionary
        return None
    
    def isUniqueVal(self, key, value):
        with open(os.path.join(self.root_path,self.csv_name)) as csv_file:
            dictionary_list = csv.DictReader(csv_file)
            for items in dictionary_list:
                if items[key]==value:
                    return True
        return False
            
        
        

In [41]:
def without_keys(dictionary: dict, exclude_keys: list) -> dict:
    return {x: dictionary[x] for x in dictionary.keys() if x not in exclude_keys}
without_keys(dictionary = {'1': 'hello', '2':'how are you', '3': 'I am fine'},exclude_keys = ['1','2'])
# hell = {'1': 'hello', '2':'how are you', '3': 'I am fine'}


{'3': 'I am fine'}

In [ ]:
manager = manageEvent(os.path.join(os.getcwd(),'event_data'),os.path.join(os.getcwd(),'event_data','event0.csv'))

In [ ]:
manager.fetchRowWithField(csv_name = 'testing.csv', fields = ['name','class','gender'],header = 'name',condition = 'sazz')

In [ ]:
manager.fetchFrom('testing.csv', ['name','class','age'])

In [42]:
def take_details():
    
    headers = ['Name', 'Student Id','Course','Contact No.','Email','Registered Date']
    name = input("Enter your Name: ")
    std_id = int(input("Enter you Student ID: "))
    
    while True:
        try:
            course = int(input("Enter your course (Input corresponding number)\n1.BIT\n2.BIBM\n3.Others\n->:"))
            if course in range(1,4):
                break
            else:
                print("Input should range between 1 and 3")
                continue
        except ValidError:
            print("Input Corresponding number")
    while True:
        try:
            contact = int(input("Phone Number: "))
            if len(str(contact))==10:
                break
            else:
                print("Not a valid Number")
                continue
        except ValueError:
            print("Only number supported")
    email = input("Enter your E-mail: ")
    date = str(datetime.date.today())
    course = 'BIT' if course == 1 else 'BIBM' if course ==2 else 'Others'
    return {headers[0]: name, headers[1]:std_id, headers[2]:course, headers[3]: contact, headers[4]: email, headers[5]:date}, headers


In [64]:
def main():
    os.chdir('C:\\jupyter_practice\\Computer_Vision_practice')
    meta_data = "testing.csv"
    event_file = "event0.csv"
    photo = take_photo()
    res = checkFace(photo, root_folder = r'.\\root_folder')
    Manager_event = manageEvent(root_path = r".\\event_data", csv_name = event_file)
    Manager_meta = manageEvent(root_path = r".\\event_data",csv_name = meta_data )
    
    if res[0]:
        print("You are registered as: ",res[1], "in our root folder")
        choice = input("Is that you? (y/n) :")
        
        if choice == 'n':
            details = take_details()
            try:
                init_folder(details[0]['Student Id'])
                save_img(details[0]['Student Id'], photo)
                Manager_meta.append(data = details[0], field_names = details[1])
                Manager_event.append(Manager_meta.fetchRowWithField(meta_data,['Name','Student Id','Course','Contact No.','Email'],header = 'Student Id',condition = details[0]['Student Id']))
            except FileExistsError:
                print("The Student ID, is already registered in our system with name: !!")    
        elif choice == 'y':
#             save_image(details[0]['Student Id'], photo)
            save_img(res[1], photo)
            if Manager_event.isUniqueVal('Student Id',str(res[1])):
                print('You are already registered in the event')
            else:            
                Manager_event.append(Manager_meta.fetchRowWithField(meta_data, ['Name','Student Id','Course','Contact No.','Email'],header = 'Student Id',condition = str(res[1])),['Name','Student Id','Course','Contact No.','Email'])

            
    else:
        details = take_details()
        try:
            init_folder(details[0]['Student Id'])
            save_img(details[0]['Student Id'], photo)
            Manager_meta.append(data = details[0], field_names = details[1])
            Manager_event.append(data = without_keys(details[0], ['Registered Date']),field_names = ['Name','Student Id','Course','Contact No.','Email'])
        except FileExistsError:
            print("Your are already registered in our system")
#         dictionary_to_be_appended = Manager_meta.fetchRowWithField(meta_data,['Name','Student Id','Course','Contact No.','Email'],header = 'Student Id', condition = details[0]['Student Id'])
#         print(dictionary_to_be_appended)
#         Manager_event.append(Manager_meta.fetchRowWithField(meta_data, ['Name','Student Id','Course','Contact No.','Email'],header = 'Student Id',condition = str(res[1])),['Name','Student Id','Course','Contact No.','Email'])
#         Manager_event.append(Manager_meta.fetchRowWithField(meta_data,['Name','Student Id','Course','Contact No.','Email'],header = 'Student Id',condition = details[0]['Student Id']),['Name','Student Id','Course','Contact No.','Email'])
        
# main()
        

You are registered as:  2358469 in our root folder
Is that you? (y/n) :y
You are already registered in our sytem


In [ ]:
# def registration():
#     photo = take_photo()
#     name = str(input("Enter Your name: "))
#     if not(checkFace(photo)):
#         save_img(name, photo)
#     if checkFace(photo):
#         print("Username: already Exist")
#         save_img(name, photo)


# def main():
#     photo = take_photo()
#     res = checkFace(photo)
#     return res
# main()
    

In [ ]:
# img = "sazz0.jpg"
# checkFace(img)
# root_folder = r'C:\\jupyter_practice\\Computer_Vision_practice\\root_folder'
# for folders in os.listdir(root_folder):
#     for images in os.listdir(root_folder + r"\\" + folders):
#         if images.endswith('.jpg'):
#             print(images)

In [ ]:
# x = [y for y in os.listdir(r'C:\\jupyter_practice\\Computer_Vision_practice\\root_folder')]

In [ ]:
# root_folder = r'C:\\jupyter_practice\\Computer_Vision_practice\\root_folder'
# for images in os.listdir(root_folder + r"\\"+ x[0]):
#     print(images)

In [ ]:
# with open('sth.csv','a', newline = '') as fp:
# #     sthDict = list(csv.DictReader(fp))
# #     testDict = {'first_name': 'Hello', 'second_name': 'Thakur', 'age': 20, 'group': 'loda'}
# #     fp.append(testDict)
#     writer = csv.DictWriter(fp, fieldnames = ['first_name', 'second_name', 'age', 'group'])
#     writer.writerow({"first_name": 'Bikalpa', 'second_name': 'Paudel', 'age': 18, 'group': 'L4CG7'})
   
    

In [65]:
# def hello():
#     with open(".\\event_data\\testing.csv") as csv_file:
#             dictionary = csv.DictReader(csv_file)
#             what_to_fetch = ['name', 'gender', 'age']
#             fetched_data = list()
#             for rows in dictionary:
#                 data = dict()
#                 for field in what_to_fetch:
#                     data[field] = rows[field]
#                 fetched_data.append(data)
#             return fetched_data
                    
#     return fetched_data

                